# Environment Initialization

In [7]:
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd
import numpy as np

from statsmodels.tsa.arima.model import ARIMA

import tensorflow as tf
from tensorflow import keras

import os
import shutil
if not os.path.exists("./report/figures/"):
  os.makedirs("./report/figures/")
if not os.path.exists("./models/"):
  os.makedirs("./models/")

# Data Import/Pre-Processing/Visualization

In [8]:
dtype = {
  "Datetime": "string",
  "solar_mw": np.float32,
  "wind-direction": np.float32, 
  "wind-speed": np.float32,
  "humidity": np.float32,
  "average-wind-speed-(period)": np.float32,
  "average-pressure-(period)": np.float32,
  "temperature": np.float32
}
solar_energy_df = pd.read_csv("./solarenergy.csv",
                              delimiter=",",
                              dtype=dtype)

solar_energy_df["Datetime"] = pd.to_datetime(solar_energy_df["Datetime"],
                                             format="%d/%m/%Y %H:%M")
solar_energy_df = solar_energy_df.set_index("Datetime").sort_index()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Raw)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_raw.svg")
plt.close("all")

solar_energy_df = solar_energy_df.dropna()
solar_energy_df = solar_energy_df.resample("1H").interpolate("linear")

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Clean)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_clean.svg")
plt.close("all")

solar_energy_df = \
  (solar_energy_df - solar_energy_df.mean()) / solar_energy_df.std()

axes = solar_energy_df.plot(figsize=(8,5), subplots=True, sharex=True)
plt.suptitle("Solar Energy Dataset (Normalized)")
axes[-1].set_xlabel("Time")
for axis in axes:
  axis.autoscale(True, "x", True)
plt.savefig("./report/figures/timeseries_norm.svg")
plt.close("all")

# Baseline Model

In [9]:
training_ratio = 0.7
training_limit = \
  solar_energy_df.index[
      int(training_ratio*solar_energy_df.shape[0])
    ]

best_rmse = np.inf
arimax_model = None

x_train_df = solar_energy_df.iloc[
                :int(training_ratio*solar_energy_df.shape[0]),
                solar_energy_df.columns != "solar_mw"
              ]
y_train_df = solar_energy_df.loc[:training_limit-pd.Timedelta(hours=1),
                                  "solar_mw"]

x_test_df = solar_energy_df.iloc[
              int(training_ratio*solar_energy_df.shape[0]):,1:
            ]
y_test_df = solar_energy_df.loc[training_limit:, "solar_mw"]

for p in [5]:
  for d in [0]:
    for q in [3]:
      try:
        new_model = ARIMA(
          endog=y_train_df,
          exog=x_train_df,
          order=(p,d,q)
        ).fit()
      except:
        continue

      new_prediction = new_model.get_forecast(
          steps=solar_energy_df.index[-1],
          exog=x_test_df).predicted_mean.to_numpy()
      
      new_rmse = np.sqrt(np.mean((new_prediction-y_test_df.to_numpy())**2))
      if new_rmse < best_rmse:
        best_rmse = new_rmse
        arimax_model = new_model

baseline_fit = arimax_model.fittedvalues

baseline_prediction = arimax_model.get_forecast(
    steps=solar_energy_df.index[-1],
    exog=x_test_df
  ).predicted_mean

print(arimax_model.summary())
p,d,q = arimax_model.specification["order"]
print(f"Baseline RMSE: {best_rmse}")
print(f"Baseline MAE: {np.abs(baseline_prediction-y_test_df.to_numpy()).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(baseline_fit, label="Fitted")
plt.plot(baseline_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"ARIMAX({p},{d},{q})")
plt.legend()
plt.savefig("./report/figures/baseline_fit.svg")
plt.close("all")

/home/sergiovaneg/miniforge3/envs/ADAML/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:               solar_mw   No. Observations:                 1544
Model:                 ARIMA(5, 0, 3)   Log Likelihood                1884.828
Date:                Fri, 17 Nov 2023   AIC                          -3737.655
Time:                        10:49:52   BIC                          -3652.181
Sample:                    05-03-2020   HQIC                         -3705.859
                         - 07-06-2020                                         
Covariance Type:                  opg                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.0943      0.123      0.764      0.445      -0.148       0.336
wind-direction                 -0.0004      0.004     -0.098      0.922      -0.

# Recursive Models

## Data Preparation

In [10]:
q=3*q
p=3*p
n_feats = len(x_train_df.columns)

def tensor_memory_reshaper(in_np:np.ndarray, out_np:np.ndarray,
                           memory:int, n_feats_internal:int = n_feats):
  in_np = in_np.reshape((-1,1,n_feats_internal))
  for _ in range(memory):
    next_np = in_np[1:,-1,:].reshape(((-1,1,n_feats_internal)))
    in_np = np.concatenate((in_np[:-1,:,:], next_np), axis=1)
  return (tf.convert_to_tensor(in_np),
          tf.convert_to_tensor(out_np[memory:]))

x_train_np = np.pad(x_train_df.to_numpy(), ((q,0),(0,0)), mode="edge")
y_train_np = np.pad(y_train_df.to_numpy(), ((q,0),), mode="edge")

x_test_np = pd.concat((x_train_df.iloc[-q:], x_test_df)).to_numpy()
y_test_np = pd.concat((y_train_df.iloc[-q:], y_test_df)).to_numpy()

x_train_tensor,y_train_tensor = \
  tensor_memory_reshaper(x_train_np, y_train_np, q)
x_test_tensor,y_test_tensor = \
  tensor_memory_reshaper(x_test_np, y_test_np, q)

class StateResetCallback(keras.callbacks.Callback):
  def on_epoch_begin(self, epoch, logs=None):
    self.model.reset_states()

## Simple RNN

In [11]:
if not os.path.exists("./models/rnn_model.keras"):
  rnn_model = keras.Sequential()
  rnn_model.add(keras.layers.SimpleRNN(p, stateful=False,
                                       batch_input_shape=(None,q+1,n_feats)))
  rnn_model.add(keras.layers.Dense(1, keras.activations.linear))

  rnn_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/rnn_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )
  rnn_model_early_stop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            patience=100,
                            restore_best_weights=False
                          )

  rnn_model.compile(optimizer="Adam", loss="mse")
  rnn_model.fit(x_train_tensor, y_train_tensor,
                shuffle=True, batch_size=64,
                epochs=1000, verbose="auto",
                validation_split=0.2,
                callbacks=[rnn_model_early_stop, StateResetCallback()])
  
  if os.path.exists("./models/rnn_model_checkpoint"):
    rnn_model = keras.models.load_model("./models/rnn_model_checkpoint")
    shutil.rmtree("./models/rnn_model_checkpoint")
  rnn_model.save("./models/rnn_model.keras")
else:
  rnn_model = keras.models.load_model("./models/rnn_model.keras")

rnn_model.reset_states()
rnn_fit = rnn_model.predict(x_train_tensor, batch_size=64).flatten()
rnn_prediction = rnn_model.predict(x_test_tensor, batch_size=64).flatten()

print(f"RNN RMSE: {np.sqrt(np.mean((rnn_prediction-y_test_tensor)**2))}")
print(f"RNN Max AE: {np.abs(rnn_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, rnn_fit, label="Fitted")
plt.plot(y_test_df.index, rnn_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"Simple RNN")
plt.legend()
plt.savefig("./report/figures/rnn_fit.svg")
plt.close("all")

11/11 [==============================] - 0s 2ms/step
RNN RMSE: 1.211552381515503
RNN Max AE: 3.1680002212524414


## LSTM

In [12]:
if not os.path.exists("./models/lstm_model.keras"):
  lstm_model = keras.Sequential()
  lstm_model.add(keras.layers.LSTM(p, stateful=False,
                                   batch_input_shape=(None,q+1,n_feats)))
  lstm_model.add(keras.layers.Dense(1, keras.activations.linear))

  lstm_model_checkpoint = keras.callbacks.ModelCheckpoint(
                            "./models/lstm_model_checkpoint",
                            monitor="loss",
                            save_best_only=True,
                            save_weights_only=False
                          )
  lstm_model_early_stop = keras.callbacks.EarlyStopping(
                            monitor="val_loss",
                            patience=100,
                            restore_best_weights=False
                          )

  lstm_model.compile(optimizer="Adam", loss="mse")
  lstm_model.fit(x_train_tensor, y_train_tensor,
                shuffle=True, batch_size=64,
                epochs=1000, verbose="auto",
                validation_split=0.2,
                callbacks=[lstm_model_early_stop, StateResetCallback()])
  
  if os.path.exists("./models/lstm_model_checkpoint"):
    lstm_model = keras.models.load_model("./models/lstm_model_checkpoint")
    shutil.rmtree("./models/lstm_model_checkpoint")
  lstm_model.save("./models/lstm_model.keras")
else:
  lstm_model= keras.models.load_model("./models/lstm_model.keras")

lstm_model.reset_states()
lstm_fit = lstm_model.predict(x_train_tensor, batch_size=64).flatten()
lstm_prediction = lstm_model.predict(x_test_tensor, batch_size=64).flatten()

print(f"LSTM RMSE: {np.sqrt(np.mean((lstm_prediction-y_test_tensor)**2))}")
print(f"LSTM Max AE: {np.abs(lstm_prediction-y_test_tensor).max()}")

plt.figure(figsize=(8,5))
plt.plot(solar_energy_df["solar_mw"],
         label="Measured")
plt.plot(y_train_df.index, lstm_fit, label="Fitted")
plt.plot(y_test_df.index, lstm_prediction, label="Estimated")
plt.autoscale(True, "x", tight=True)
plt.title(f"LSTM")
plt.legend()
plt.savefig("./report/figures/lstm_fit.svg")
plt.close("all")

11/11 [==============================] - 0s 2ms/step
LSTM RMSE: 1.4091228246688843
LSTM Max AE: 3.301408052444458
